# 1. Data import and processing

## 1.1 引入数据集，生成dataframe

In [1]:
import pandas as pd
import numpy as np

data_path ='./dataset_new.csv'
columns_name = ('date', 'time', 'sensor_id', 'value', 'activity', 'status') # status是begin或end
df = pd.read_csv(data_path, names = columns_name)

In [2]:
df.dtypes

date         object
time         object
sensor_id    object
value        object
activity     object
status       object
dtype: object

In [3]:
'''
#一些测试
df.iloc[:,:4]
df_first4Attr = df.iloc[:,:4]
df[df_first4Attr.isnull().values==True]
'''

'\n#一些测试\ndf.iloc[:,:4]\ndf_first4Attr = df.iloc[:,:4]\ndf[df_first4Attr.isnull().values==True]\n'

In [4]:
df['sensor_id'].unique()

array(['M35', 'AD1-A', 'M34', 'M32', 'M33', 'M36', 'M30', 'M29', 'M37',
       'M38', 'M39', 'M41', 'M40', 'M49', 'M46', 'M50', 'M44', 'M47',
       'M48', 'M43', 'M28', 'M42', 'M27', 'M26', 'M25', 'M24', 'M21',
       'M19', 'M18', 'M17', 'M16', 'M15', 'D09', 'M14', 'M13', 'M08',
       'M07', 'M02', 'M45', 'M51', 'D08', 'AD1-B', 'AD1-C', 'D10', 'I03',
       'M06', 'M09', 'M10', 'M01', 'D12', 'L04', 'M23', 'M31', 'D15',
       'D07', 'D14', 'M11', 'M05', 'M03', 'M04', 'M22', 'M12', 'M20',
       'L11', 'D05', 'D03', 'L10', 'L06', 'L13', 'L12', 'L09'],
      dtype=object)

## 1.2 判断缺失值

In [5]:
df.dropna(axis=0, how='all', inplace=True) ## 删去了两行空白行
df

,date,time,sensor_id,value,activity,status
0,02/02/2009,7:15:17,M35,ON,R1_Bed_to_Toilet,begin
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN
2,02/02/2009,7:15:23,M35,OFF,NaN,NaN
3,02/02/2009,7:15:23,M35,ON,NaN,NaN
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,OFF,NaN,NaN
138626,04/04/2009,7:27:47,M48,ON,NaN,NaN
138627,04/04/2009,7:27:50,M47,ON,NaN,NaN
138628,04/04/2009,7:27:59,M47,OFF,NaN,NaN


## 1.3 合并date和time

In [6]:
# df.drop(df.tail(1).index, inplace = True) #从尾部去掉 1 行，这一行是异常行

In [7]:
# df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], format='%d/%m/%Y %H:%M:%S')
df["datetime"] = pd.to_datetime(df["date"] + " "+ df["time"], dayfirst=True)

# df["datetime"]


In [8]:
# df = df.set_index("datetime") #千万不能设datetime为index! 因为datetime有重复值！！！妈的，debug半个多小时。
df

,date,time,sensor_id,value,activity,status,datetime
0,02/02/2009,7:15:17,M35,ON,R1_Bed_to_Toilet,begin,2009-02-02 07:15:17
1,02/02/2009,7:15:21,AD1-A,2.82231,NaN,NaN,2009-02-02 07:15:21
2,02/02/2009,7:15:23,M35,OFF,NaN,NaN,2009-02-02 07:15:23
3,02/02/2009,7:15:23,M35,ON,NaN,NaN,2009-02-02 07:15:23
4,02/02/2009,7:15:28,AD1-A,2.79926,NaN,NaN,2009-02-02 07:15:28
...,...,...,...,...,...,...,...
138625,04/04/2009,7:26:16,M48,OFF,NaN,NaN,2009-04-04 07:26:16
138626,04/04/2009,7:27:47,M48,ON,NaN,NaN,2009-04-04 07:27:47
138627,04/04/2009,7:27:50,M47,ON,NaN,NaN,2009-04-04 07:27:50
138628,04/04/2009,7:27:59,M47,OFF,NaN,NaN,2009-04-04 07:27:59


In [9]:
# # 检查时间都是前一个<=后一个
# for a, b in zip(df.datetime, df.datetime[1:]):
#     if(a>b):
#         print(a)
#
# # all(a <= b for a, b in zip(df.index, df.index[1:]))

## 1.4 检查R1以及公共的各种行为不存在嵌套

In [20]:
df.activity.unique()

array(['R1_Bed_to_Toilet', nan, 'R1_Personal_Hygiene', 'R2_Bed_to_Toilet',
       'Meal_Preparation', 'R2_Personal_Hygiene', 'Watch_TV', 'R1_Work',
       'R2_Work', 'R1_Sleep', 'R2_Sleep', 'Wash_Bathtub', 'Clean',
       'Study'], dtype=object)

In [23]:
R1_activities = ["R1_Bed_to_Toilet", "R1_Personal_Hygiene", "Meal_Preparation", "Watch_TV", "R1_Work", "R1_Sleep", "Wash_Bathtub", "Clean", "Study"]
def check_if_R1_sleep_nesting(df):
    stack = []
    # cnt = 0
    for index,row in df.iterrows():
        if (df.loc[index, 'activity'] in R1_activities):
            if row.status == 'begin':
                stack.append(row.activity)
                # print(stack)
            elif row.status == 'end':
                stack.pop()
                # print(stack)
            if(len(stack) > 1):
                print("Alarm !")

        # # 每100次打印一次栈的状态
        # cnt += 1
        # if cnt % 100 == 0:
        #     print(stack)
    print("The final stack value:", stack)

check_if_R1_sleep_nesting(df)

# 发现有一些R1的个人行为和公共行为是存在嵌套的，不过不影响后续的提取。
# R1_sleep 是不存在嵌套的

Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
Alarm !
The final stack value: []


## 1.5 通过原有的dateframe 生成新的dataframe包含：activity, start_time, end_time, time_span

In [27]:
def create_transformed_df(old_df, R1_activities):
    new_df = pd.DataFrame(columns=['activity', 'start_time', 'end_time', 'time_span'])

    # cnt = 0
    for activity_name in R1_activities:
        # 初始化
        start_time = datetime.time(0, 0, 0)
        end_time = datetime.time(0,0,0)
        time_span = 0
        for index,row in old_df.iterrows():
            if row.status == 'begin':
                if row.activity == activity_name:
                    print(activity_name)
                    # stack.append(row.activity)
                    start_time = row.datetime
            elif row.status == 'end':
                if row.activity == activity_name:
                    # 向new_df增加数据
                    # del(stack[stack.index(row.activity)])
                    end_time = row.datetime
                    time_span = (end_time - start_time).seconds
                    tmp_df = pd.DataFrame({'activity': [activity_name],
                    'start_time' : [start_time],
                    'end_time' : [end_time],
                    'time_span' : [time_span]})

                    new_df = pd.concat([new_df, tmp_df], ignore_index = True, axis = 0)


    return new_df



In [28]:
new_df = create_transformed_df(df, R1_activities)

R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toilet
R1_Bed_to_Toil

In [29]:
new_df

,activity,start_time,end_time,time_span
0,R1_Bed_to_Toilet,2009-02-02 07:15:17,2009-02-02 07:21:04,347
1,R1_Bed_to_Toilet,2009-02-03 07:29:34,2009-02-03 07:35:48,374
2,R1_Bed_to_Toilet,2009-02-04 07:26:00,2009-02-04 07:40:03,843
3,R1_Bed_to_Toilet,2009-02-05 06:51:44,2009-02-05 07:12:46,1262
4,R1_Bed_to_Toilet,2009-02-06 07:33:50,2009-02-06 07:53:46,1196
...,...,...,...,...
353,Study,2009-03-10 19:41:14,2009-03-10 22:41:16,10802
354,Study,2009-03-11 19:39:57,2009-03-11 22:09:45,8988
355,Study,2009-03-23 20:35:02,2009-03-23 22:23:05,6483
356,Study,2009-03-24 08:24:15,2009-03-24 08:36:44,749
